Importing the libraries

In [31]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

 # map rendering library
!pip install folium
import folium

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Scraping the web and preparing the table.

In [32]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    

    if row != [] and row[1] != "Not assigned":


            if "Not assigned" in row[2]: 
                 row[2] = row[1]
            res.append(row)
           

df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])


df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")


indexNum = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNum, inplace = True)

Importing the geographical coordinates table of each "M" group postal code of Toronto/Canada.

In [33]:
df_geo_coor = pd.read_csv("https://cocl.us/Geospatial_data")

# if you wish the look the 
# df_geo_coor.head()

Merging the tables

In [34]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')

# removing the mutual "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)

## Latitude and longitude values of Toronto

In [35]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


## Creating the map of Toronto, using latitude and longitude values.

In [44]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

## Adding markers.

In [43]:
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Dark blue',
        fill=True,
        fill_color='Blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

 ## Listing the boroughs that contain the word "Toronto" of Toronto City's. 

In [40]:
# "denc" = [D]owntown Toronto, [E]ast Toronto, [N]orth Toronto, [C]entral Toronto
df_toronto_denc = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_denc.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


## The map of the boroughs that contain the word "Toronto" of Toronto City's.

In [42]:
map_toronto_denc = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_toronto_denc['Latitude'], 
        df_toronto_denc['Longitude'], 
        df_toronto_denc['Borough'], 
        df_toronto_denc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_denc)  

map_toronto_denc